In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. Sicherstellen, dass wir im richtigen Ordner sind
%cd /content/drive/MyDrive/AI-Projekt

# 2. Status prüfen (du solltest sehen, dass Kreisel.ipynb geändert wurde)
!git status

# 3. Änderungen hinzufügen und hochladen
!git add .
!git commit -m "Kreisel Notebook aktualisiert"
!git push origin woche2

/content/drive/MyDrive/AI-Projekt
On branch woche2
nothing to commit, working tree clean
On branch woche2
nothing to commit, working tree clean
Everything up-to-date
